In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
tree = ET.parse('network.xml')
root = tree.getroot()

In [3]:
cols=[ 'WKT','linkId','width']
rows = []

In [4]:
## append row
link_to_width = {}
for links in root.findall('links'):
    for link in links.findall('link'):
        for attributes in link.findall('attributes'):
            for attribute in attributes.findall('attribute'):
                if attribute.get('name') == 'width' :
                    sum_width = 0
                    list_width   = attribute.get('width').split(' ')
                    for width in list_width:
                        if (width != ''):
                            sum_width = sum_width + float (width) 
                    avg_width = sum_width / len (list_width)
                    link_to_width [link.get('linkId')]=str(avg_width)

In [5]:
current_link = None
current_geometry = None 
for nodes in root.findall('nodes'):
    for node in nodes.findall('node'):
        if current_link is None or node.get('linkId') != current_link:
            current_link= node.get('linkId')
            if current_geometry is None:
                current_geometry= 'MULTILINESTRING ((' + node.get('geometry')
            else:
                current_geometry= current_geometry + '))'
                
                rows.append({'WKT':current_geometry,'linkId':current_link, 'width':link_to_width[current_link]})
    
                current_geometry= 'MULTILINESTRING ((' + node.get('geometry')
        else:
            current_geometry= current_geometry +','+ node.get('geometry')
    
        
    rows.append({'WKT':current_geometry+'))','linkId':current_link, 'width':link_to_width[current_link]})
                    
        

In [6]:
df = pd.DataFrame(rows, columns=cols)

# Writing dataframe to csv
df.to_csv('output.csv',index=False)